In [11]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


In [4]:
load_dotenv()

True

In [5]:
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.5)

/home/miller/.local/share/virtualenvs/lang_chain_studies-WWOOpBwl/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [7]:
prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}'
)

prompt = prompt_template.format(adjective='happy', content='Portugal')

response = model.invoke(prompt)

print(response)



Why did the Portuguese man go to the doctor?

Because he was feeling a little Porto-sick!


## ChatPromptTemplate: The prompt to chat models is a list of chat messages

In [1]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpfull AI bot. Your name is {name}.'),
        ('human', 'Hello how are you doing'),
        ('ai', 'I\'m doing well, thanks!'),
        ('human', '{user_input}')
    ]
)

prompt = chat_template.format_messages(name='Bob', user_input='What is your name and what are you good at?')

response = model.invoke(prompt)

print(response)


AI: My name is Bob and I am an AI designed to assist and provide helpful responses to users. I am also good at processing and analyzing large amounts of data quickly and accurately. Is there something specific you would like me to help you with?


## Various ways of formatting System/Human/Ai prompts

In [8]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assiatnt that re-writes the user's text to"
                "sound more upbeat"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

prompt = chat_template.format_messages(text="I don't like eating tasty things")

response = model.invoke(prompt)

print(response)



System: I love indulging in delicious treats!


## Providing a context along the prompt

In [15]:
prompt = """Answer teh question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".vscode/

Context: Lage Language models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models 
can be acessed via Hugging Face's transformers library, via OpenAI
using the 'openai' library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer:"""

print(model.invoke(prompt))

 Hugging Face's transformers library, OpenAI's 'openai' library, and Cohere's `cohere` library.


## Langchain FewShotPromptTemplate

In [9]:
from langchain import FewShotPromptTemplate

In [19]:
# create examples
examples = [
    {
        'query': 'How are you?',
        'answer': 'I can\'t complain bu sometimes I still do.'
    },
    {
        'query': 'What times is it?',
        'answer': 'It\'s to get a watch.'
    }
]

example_template = """
User: {query}
AI: {answer}
"""

In [23]:
# create a prompt exmaple from above template
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the preffix is our instructions
prefix = """The following are experts from conversations with AI assitant.
The assitant is typically sarcastic anwitty, producing
creative and funny responses to the users questions. Here are some 
examples:"""

# and the suffix our user input and output indicator
suffix = """
User: {query}
AI:"""

In [26]:
few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator='\n\n'
)

query = 'Qhat movie should Iwatch evening?'

print(few_shot_template.format(query=query))

The following are experts from conversations with AI assitant.
The assitant is typically sarcastic anwitty, producing
creative and funny responses to the users questions. Here are some 
examples:


User: How are you?
AI: I can't complain bu sometimes I still do.



User: What times is it?
AI: It's to get a watch.



User: Qhat movie should Iwatch evening?
AI:


In [27]:
chain = few_shot_template | model

chain.invoke({'query': query})

' How about "The Shawshank Redemption"? It\'s a classic and I\'m sure you haven\'t seen it for the 100th time yet.'

In [10]:
template = '''You are a experience virologist.
Write a few sentences about the following virus {virus} in {language}.'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='german')

prompt

'Tou are a experience virologist.\nWrite a few sentences about the following virus hiv in german.'

In [14]:
llm = ChatOpenAI(name='gpt-3.5-turbo', temperature=0)

output = llm.invoke(prompt)

print(output)

content='HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt und zu AIDS führen kann. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV zu informieren, um die Verbreitung dieser lebensbedrohlichen Krankheit einzudämmen.' response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 30, 'total_tokens': 156}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-69de31e6-404d-48ee-96e4-5b28ac7d5ab2-0' usage_metadata={'input_tokens': 30, 'output_tokens': 126, 'total_tokens': 156}


In [16]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage


chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the json format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='20', area='Europe')

messages

[SystemMessage(content='You respond only in the json format.'),
 HumanMessage(content='Top 20 countries in Europe by population.')]

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

output = llm.invoke(messages)

print(output)

content='{\n    "1": "Germany",\n    "2": "Russia",\n    "3": "Turkey",\n    "4": "France",\n    "5": "United Kingdom",\n    "6": "Italy",\n    "7": "Spain",\n    "8": "Ukraine",\n    "9": "Poland",\n    "10": "Romania",\n    "11": "Netherlands",\n    "12": "Belgium",\n    "13": "Czech Republic",\n    "14": "Greece",\n    "15": "Portugal",\n    "16": "Sweden",\n    "17": "Hungary",\n    "18": "Belarus",\n    "19": "Austria",\n    "20": "Serbia"\n}' response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 28, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-af2d831e-e21b-46b9-8bbe-d8b6b2b27a34-0' usage_metadata={'input_tokens': 28, 'output_tokens': 156, 'total_tokens': 184}
